In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from dotenv import load_dotenv
load_dotenv()
import os
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
HUGGING_FACE_API_KEY = os.getenv('HUGGING_API_KEY')

In [3]:
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

#initialize client
pc =Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name =index_name,
        dimension=384, #dimension of dense factor
        metric='dotproduct', #sparse values supported only dot product
        spec =ServerlessSpec(cloud='aws',region='us-east-1')
    )


In [4]:
index=pc.Index(index_name)
index

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model='all-MiniLm-L6-v2')
embeddings


HuggingFaceEmbeddings(model_name='all-MiniLm-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

In [8]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]
bm25_encoder.fit(sentences)

bm25_encoder.dump("bm25_values.json")

bm25_encoder = BM25Encoder().load("bm25_values.json")


100%|██████████| 3/3 [00:00<00:00, 19.30it/s]


In [9]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [10]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLm-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000017607E277D0>, index=<pinecone.data.index.Index object at 0x00000175CDCC0B10>)

In [11]:
retriever.add_texts(
    [
           "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans", 
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.16s/it]


In [16]:
retriever.invoke("What city did i visit 2023")

[Document(metadata={'score': 0.490044475}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.372445524}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.317856967}, page_content='In 2021, I visited New Orleans')]